In [73]:
%%time
import gensim
from gensim.models import word2vec
import re
import pandas as pd
import numpy as np
from typing import Dict, Tuple
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from nltk import WordPunctTokenizer
from nltk.corpus import stopwords
import nltk
import psycopg2
nltk.download('stopwords')

CPU times: total: 62.5 ms
Wall time: 276 ms


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [74]:
con = psycopg2.connect(
 database="postgres", 
 user="postgres", 
 password="********", 
 host="localhost", 
 port="****"
)

In [45]:
%%time
df_load = pd.read_csv("archive/spam_ham_dataset.csv", sep=',')

CPU times: total: 125 ms
Wall time: 124 ms


In [46]:
df_load.head()

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0


In [47]:
df_class = df_load.iloc[999:]

In [48]:
df_class.head()

,Unnamed: 0,label,text,label_num
999,4446,spam,Subject: vicodin for you . cheap .\r\nyou need...,1
1000,305,ham,"Subject: logistics enom team\r\nhal , as per o...",0
1001,688,ham,Subject: so union\r\ntville\r\n5031 at tville ...,0
1002,1465,ham,Subject: september 2000 availabilities\r\nthis...,0
1003,1459,ham,Subject: potential list fo 9 / 00\r\ndaren :\r...,0


In [41]:
%%time
# Подготовим корпус
corpus = []
stop_words = stopwords.words('english')
tok = WordPunctTokenizer()
for line in df_class['text'].values:
    line1 = line.strip().lower()
    line1 = re.sub("[^a-zA-Z]"," ", line1)
    text_tok = tok.tokenize(line1)
    text_tok1 = [w for w in text_tok if not w in stop_words]
    corpus.append(text_tok1)

CPU times: total: 1.47 s
Wall time: 1.46 s


In [42]:
corpus[:5]

[['subject',
  'vicodin',
  'cheap',
  'need',
  'vicodin',
  'get',
  'need',
  'wait',
  'longer',
  'unique',
  'chance',
  'save',
  'medications',
  'saving',
  'boosting',
  'health',
  'courtesy',
  'wristwatch',
  'consensus',
  'chapel',
  'baltimorean',
  'halfway',
  'debacle',
  'torsion',
  'hey',
  'song',
  'staminate',
  'spring',
  'teledyne',
  'freakish',
  'becker',
  'basswood',
  'ben',
  'aphelion',
  'mileage',
  'moulton',
  'poetic',
  'interstitial',
  'godson',
  'arctic',
  'swelter',
  'bust',
  'egress',
  'nonsensic',
  'sectarian',
  'caloric',
  'qualify',
  'embedding',
  'expand',
  'cupboard',
  'splintery',
  'macedonia',
  'inheritance',
  'polyhymnia',
  'conceal',
  'deathward',
  'fingernail',
  'suppressor',
  'befallen',
  'firewall',
  'kappa',
  'spiral',
  'army',
  'carcinoma',
  'fermat',
  'lillian',
  'monteverdi',
  'salisbury',
  'suck',
  'scot',
  'tendency',
  'basin',
  'meteoritic',
  'dustbin',
  'climax',
  'anorexia',
  'aver

In [43]:
%%time
#Обучаем модель word2vec на нашем корпусе
model_dz = word2vec.Word2Vec(corpus, workers=4, min_count=10, window=10, sample=1e-3)

CPU times: total: 3.48 s
Wall time: 1.23 s


In [44]:
# Проверим, что модель обучилась
print(model_dz.wv.most_similar(positive=['find'], topn=5))

[('look', 0.9450597167015076), ('want', 0.9429246783256531), ('leave', 0.9421932697296143), ('someone', 0.9355387091636658), ('able', 0.9337681531906128)]


In [57]:
%%time
class EmbeddingVectorizer(object):
    '''
    Для текста усредним вектора входящих в него слов
    '''
    def __init__(self, model):
        self.model = model
        self.size = model.vector_size

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([np.mean(
            [self.model[w] for w in words if w in self.model] 
            or [np.zeros(self.size)], axis=0)
            for words in X])
def accuracy_score_for_classes(
    y_true: np.ndarray, 
    y_pred: np.ndarray) -> Dict[int, float]:
    """
    Вычисление метрики accuracy для каждого класса
    y_true - истинные значения классов
    y_pred - предсказанные значения классов
    Возвращает словарь: ключ - метка класса, 
    значение - Accuracy для данного класса
    """
    # Для удобства фильтрации сформируем Pandas DataFrame 
    d = {'t': y_true, 'p': y_pred}
    df = pd.DataFrame(data=d)
    # Метки классов
    classes = np.unique(y_true)
    # Результирующий словарь
    res = dict()
    # Перебор меток классов
    for c in classes:
        # отфильтруем данные, которые соответствуют 
        # текущей метке класса в истинных значениях
        temp_data_flt = df[df['t']==c]
        # расчет accuracy для заданной метки класса
        temp_acc = accuracy_score(
            temp_data_flt['t'].values, 
            temp_data_flt['p'].values)
        # сохранение результата в словарь
        res[c] = temp_acc
    return res
def print_accuracy_score_for_classes(
    y_true: np.ndarray, 
    y_pred: np.ndarray):
    """
    Вывод метрики accuracy для каждого класса
    """
    accs = accuracy_score_for_classes(y_true, y_pred)
    if len(accs)>0:
        print('Метка \t Accuracy')
    for i in accs:
        print('{} \t {}'.format(i, accs[i]))
def spam(v, c):
    model = Pipeline(
        [("vectorizer", v), 
         ("classifier", c)])
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print_accuracy_score_for_classes(y_test, y_pred)
    return model

CPU times: total: 0 ns
Wall time: 0 ns


In [53]:
%%time
# Обучающая и тестовая выборки
boundary = 3000
X_train = corpus[:boundary] 
X_test = corpus[boundary:]
y_train = df_class.label.values[:boundary]
y_test = df_class.label.values[boundary:]

CPU times: total: 0 ns
Wall time: 0 ns


In [111]:
%%time
word2vecKNN = spam(EmbeddingVectorizer(model_dz.wv), KNeighborsClassifier(4))

Метка 	 Accuracy
ham 	 0.9741379310344828
spam 	 0.9
CPU times: total: 1.14 s
Wall time: 1.06 s


In [72]:
%%time
exmpl = []
for i in range(100):
    val = df_load.iloc[i, 2]
    val1 = val.strip().lower()
    val1 = re.sub("[^a-zA-Z]"," ", val1)
    val1_tok = tok.tokenize(val1)
    val1_tok1 = [w for w in val1_tok if not w in stop_words]
    exmpl.append(val1_tok1)
print(word2vecKNN.predict(exmpl))

['ham' 'ham' 'ham' 'spam' 'ham' 'ham' 'spam' 'spam' 'ham' 'ham' 'spam'
 'ham' 'ham' 'spam' 'ham' 'spam' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham'
 'ham' 'spam' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham'
 'spam' 'ham' 'ham' 'ham' 'ham' 'spam' 'ham' 'ham' 'ham' 'ham' 'ham'
 'spam' 'spam' 'spam' 'ham' 'ham' 'ham' 'ham' 'spam' 'ham' 'ham' 'ham'
 'ham' 'ham' 'ham' 'spam' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'spam'
 'ham' 'spam' 'spam' 'ham' 'spam' 'spam' 'spam' 'ham' 'ham' 'ham' 'ham'
 'ham' 'ham' 'ham' 'ham' 'ham' 'spam' 'ham' 'ham' 'ham' 'ham' 'spam'
 'spam' 'spam' 'spam' 'spam' 'spam' 'ham' 'ham' 'spam' 'ham']
CPU times: total: 125 ms
Wall time: 133 ms


In [88]:
%%time
t = 'class'
day1 = '2022-09-17'
day2 = '2022-09-19'
sql_class = "SELECT id, text FROM "+t+ " WHERE date BETWEEN '" + str(day1)+"' AND '"+ str(day2)+"'"
print(sql_class)

SELECT id, text FROM class WHERE date BETWEEN '2022-09-17' AND '2022-09-19'
CPU times: total: 0 ns
Wall time: 0 ns


In [106]:
%%time
df_class = pd.read_sql_query(sql_class, con=con)

CPU times: total: 0 ns
Wall time: 6 ms


<timed exec>:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.


In [91]:
df_class.head()

,id,text
0,820,Subject: get that new car 8434\r\npeople nowth...
1,821,"Subject: deals 141877 , 141883 , and 141884 ( ..."
2,822,"Subject: tenaska iv\r\nrick ,\r\nattached are ..."
3,823,"Subject: enron / hpl actuals for january 3 , 2..."
4,824,Subject: electronic pay stubs\r\nget ready . b...


In [92]:
%%time
exmpl = []
for i in range(29):
    val = df_class.iloc[i, 1]
    val1 = val.strip().lower()
    val1 = re.sub("[^a-zA-Z]"," ", val1)
    val1_tok = tok.tokenize(val1)
    val1_tok1 = [w for w in val1_tok if not w in stop_words]
    exmpl.append(val1_tok1)
print(word2vecKNN.predict(exmpl))

['spam' 'ham' 'ham' 'ham' 'ham' 'spam' 'spam' 'spam' 'ham' 'ham' 'ham'
 'ham' 'ham' 'ham' 'ham' 'spam' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham'
 'ham' 'spam' 'ham' 'ham' 'spam' 'ham']
CPU times: total: 62.5 ms
Wall time: 44 ms


In [107]:
df_class["length"] = ""
for i in range(len(df_class)):
    df_class.iloc[i, 2] = len(df_class.iloc[i,1])

In [108]:
df_class.head()

,id,text,length
0,820,Subject: get that new car 8434\r\npeople nowth...,174
1,821,"Subject: deals 141877 , 141883 , and 141884 ( ...",648
2,822,"Subject: tenaska iv\r\nrick ,\r\nattached are ...",1182
3,823,"Subject: enron / hpl actuals for january 3 , 2...",107
4,824,Subject: electronic pay stubs\r\nget ready . b...,1078


In [110]:
x = df_class["length"].sum()
print(x)

45728
